In [4]:
import base64
import time
import datetime
import requests
import json
import os, random, csv
from PIL import Image
from openai import OpenAI
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# CONFIGURATION
API_KEY = ""  # replace with your key
client = OpenAI(api_key=API_KEY)

base_path = "/content/drive/My Drive/CueTask"
stimuli_path = os.path.join(base_path, "stimuli")
retrocue_path = os.path.join(base_path, "retrocue_stimuli")
output_path = os.path.join(base_path, "generated_trials")
os.makedirs(output_path, exist_ok=True)

# Trial settings
N_TRIALS = 5 #decide the number of trials you want to run
grid_size = (600, 600)
cell_size = (300, 300)
positions = {
    "top_left": (0, 0),
    "top_right": (300, 0),
    "bottom_left": (0, 300),
    "bottom_right": (300, 300)
}


#   HELPER FUNCTIONS

def safe_completion_call(client, messages, max_retries=5):
    """Wrapper to retry if hitting rate limits."""
    for attempt in range(max_retries):
        try:
            return client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages
            )
        except Exception as e:
            if "rate_limit" in str(e).lower():
                wait_time = (attempt + 1) * 2
                print(f"Rate limit hit. Waiting {wait_time}s...")
                time.sleep(wait_time)
            else:
                raise
    raise Exception("Failed after multiple retries due to rate limits.")


def generate_trial(trial_num, run_folder):
    """Generate one trial grid with 4 unique color-shape stimuli and one retrocue."""
    # Select 4 unique stimuli
    all_stimuli = [f for f in os.listdir(stimuli_path) if f.endswith(".png")]
    color_shape_pairs = [f.replace(".png", "").split("_") for f in all_stimuli]

    used_colors = set()
    used_shapes = set()
    chosen_stimuli = []

    random.shuffle(color_shape_pairs)
    for color, shape in color_shape_pairs:
        if color not in used_colors and shape not in used_shapes:
            chosen_stimuli.append(f"{color}_{shape}.png")
            used_colors.add(color)
            used_shapes.add(shape)
        if len(chosen_stimuli) == 4:
            break

    if len(chosen_stimuli) < 4:
        raise ValueError("Not enough unique color-shape combinations to fill the grid!")

    # Create a grid
    canvas = Image.new("RGB", grid_size, (255, 255, 255))
    loc_mapping = {}

    for pos_name, file_name in zip(positions.keys(), chosen_stimuli):
        stim_img = Image.open(os.path.join(stimuli_path, file_name)).resize(cell_size)
        canvas.paste(stim_img, positions[pos_name])
        loc_mapping[pos_name] = file_name

    # Randomly choose query stimulus
    query_loc = random.choice(list(loc_mapping.keys()))
    query_stim = loc_mapping[query_loc]
    query_color, query_shape = query_stim.replace(".png", "").split("_")

    # Decide condition (match vs no-match)
    match = random.choice([True, False])
    retrocue_files = [f for f in os.listdir(retrocue_path) if f.endswith(".png")]
    cue_type = random.choice(["object", "feature"])  # could add "color"/"shape" split later

    # Pick retrocue stimulus
    if match:
        # YES condition — same color/shape/object as query at same location
        if cue_type == "object":
            retrocue_candidates = [f"obj_{query_color}_{query_shape}.png"]
        else:
            retrocue_candidates = [f"{query_color}.png", f"{query_shape}.png"]
        retrocue_file = random.choice([f for f in retrocue_candidates if f in retrocue_files])
        cue_location = query_loc
        correct_answer = "yes"
    else:
        # NO condition — must differ in both color and shape, and different location
        invalid_terms = {query_color, query_shape, f"obj_{query_color}_{query_shape}"}
        valid_retrocues = [f for f in retrocue_files if not any(term in f for term in invalid_terms)]
        if not valid_retrocues:
            raise ValueError("No suitable 'no' retrocue found! Check file naming consistency.")
        retrocue_file = random.choice(valid_retrocues)

        # ensure cue not in query position
        cue_locations = [loc for loc in positions.keys() if loc != query_loc]
        cue_location = random.choice(cue_locations)
        correct_answer = "no"

    # Save grid image directly into run folder
    trial_filename = f"trial_{trial_num}.png"
    trial_path = os.path.join(run_folder, trial_filename)
    canvas.save(trial_path)

    # Step 7: Return trial info
    return {
        "trial_image": trial_path,
        "retrocue_file": retrocue_file,
        "query_location": query_loc,
        "query_stimulus": query_stim,
        "retrocue_stimulus": retrocue_file,
        "correct_answer": correct_answer,
        "cue_type": cue_type,
        "cue_location": cue_location
    }


def encode_image_base64(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")


def run_trial(trial_info):
    """Send trial image + retrocue to GPT and get response."""
    grid_b64 = encode_image_base64(trial_info["trial_image"])
    retrocue_path_full = os.path.join(retrocue_path, trial_info["retrocue_file"])
    retrocue_b64 = encode_image_base64(retrocue_path_full)

    response = safe_completion_call(client, [
        {"role": "system", "content": "You are performing a visual working memory task."},
        {"role": "user", "content": [
            {"type": "text", "text": "Look at this memory array:"},
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{grid_b64}"}},
            {"type": "text", "text": "Now, here is a retrocue:"},
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{retrocue_b64}"}},
            {"type": "text", "text": f"Was the retrocue stimulus located in the {trial_info['query_location'].replace('_','-')} position? Answer 'yes' or 'no'."}
        ]}
    ])

    gpt_answer = response.choices[0].message.content.strip().lower()
    gpt_answer = gpt_answer.replace(".", "").replace("!", "").strip()
    return gpt_answer



# main function

def main():
    run_id = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    run_folder = os.path.join(output_path, f"run_{run_id}")
    os.makedirs(run_folder, exist_ok=True)

    results_file = os.path.join(run_folder, f"results_{run_id}.csv")

    with open(results_file, "w", newline="") as csvfile:
        fieldnames = ["trial", "trial_image", "retrocue_file", "query_location",
                      "query_stimulus", "retrocue_stimulus", "correct_answer",
                      "gpt_response", "accuracy", "cue_type", "cue_location"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for t in range(1, N_TRIALS + 1):
            trial_info = generate_trial(t, run_folder)
            gpt_answer = run_trial(trial_info)
            accuracy = (gpt_answer == trial_info["correct_answer"])

            trial_info.update({
                "trial": t,
                "gpt_response": gpt_answer,
                "accuracy": accuracy
            })
            writer.writerow(trial_info)
            print(f"Trial {t}: GPT={gpt_answer}, Correct={trial_info['correct_answer']}, Accuracy={accuracy}")

    print(f"\n Experiment finished! Results saved in {run_folder}")

if __name__ == "__main__":
    main()




Mounted at /content/drive
Trial 1: GPT=yes, Correct=yes, Accuracy=True
Trial 2: GPT=no, Correct=yes, Accuracy=False
Trial 3: GPT=no, Correct=no, Accuracy=True
Trial 4: GPT=no, Correct=no, Accuracy=True
Trial 5: GPT=no, Correct=no, Accuracy=True

✅ Experiment finished! Results saved in /content/drive/My Drive/CueTask/generated_trials/run_20251003_233102
